# Importing the libraries

Importing the sqlite3, json and codecs libraries

In [1]:
import sqlite3
import json
import codecs
from IPython.display import HTML

# Connecting to a database and setting up a cursor

The following command will open a connection to the existing SQLLite database. If the given database name does not exist then this call will create the database.

In [2]:
conn = sqlite3.connect('geodata.sqlite')
print conn                      # This returns a connection object, since the database is loaded successfully.
cur = conn.cursor()
print cur                       # This returns a cursor object which will be used throughout the database.

# Quering the database and creating a json file to write the information

In [3]:
# Selecting all the data from the locations table present in the database
cur.execute('SELECT * FROM Locations')
print cur                       # cursor handle returned with updated information

Following line will create a .json file named **location.js** with **write(w)** permissions and **UTF-8** encoding format. codecs will store the object which handles this newly created file into a variable named **fhand**. This object is then used to write data which is queried from the database **geodata.sqlite** into the file **location.js**. 

In [4]:
fhand = codecs.open('location.js','w', "utf-8")
fhand.write("myData = [\n")

# Updating the location.js file with the geographic information

Following loop will take each row of data present inside the cursor handler **cur** (which apparently is the entire **Locations** table present inside the loaded database) and update the json file with the geographic details of all universities around the world 

In [5]:
count = 0                       # creating a counter variable         
for row in cur:
    data = str(row[0])          # row contains a single row data entry in the table.  
    data = str(row[1])          # this contains extensive geographic information pertaining to each university.
    
    # This is used to deserialize the json document code into Python format
    try: js = json.loads(str(data))
    except: continue
        
    # if status of each college information provided in the json document is not 'OK' then continue.    
    if not('status' in js and js['status'] == 'OK') : continue
        
    # acquire latitude and longitude of each university    
    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    
    # if latitude and longitude are 0 then continue
    if lat == 0 or lng == 0 : continue
        
    # acquire the address of each university.
    where = js['results'][0]['formatted_address']
    where = where.replace("'","")
    
    # writing the data acquired from the database into the json file
    try :
        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "[" + str(lat) + "," + str(lng) + ", '" + where + "']"
        fhand.write(output)
    except:
        continue

# Closing file and cursor handlers

Having successfully written the data into location.js file, we close the file handler and the cursor handler used for querying the database.

In [6]:
fhand.write("\n];\n")

# closing the cursor handler and json object handler
cur.close()
fhand.close()

print count, "records written to location.js"
print "Open where.html to view the data in a browser"

193 records written to location.js
Open where.html to view the data in a browser


# Google Maps with the location of all universities present in the database

Universities added using geoload.py
1. K. J. Somaiya College Of Engineering, Vidyavihar, Ghatkophar
2. Rungta College of Engineering and Technology, Kohka-Kurud Road, Bhilai
3. MKSSS's Cummins College Of Engineering for woman, Karve Nagar, Pune
4. Shri Ramdeobaba College of Engineering and Management, Katol Road, Nagpur

In [7]:
HTML('<iframe src = "where.html", width = 100%, height = "500"></iframe>')